This notebook contains examples of using model index and OPC UA functions to download inverters, strings set and trackers data of the site 'EG-AS' from the model index and OPC UA api servers. The aggregated historical data downloaded in the data folder and cell execution time can be noted for each request.

### Import Libraries

In [ ]:
# Import the required packeages
import pandas as pd
from typing import List, Dict
import sys
import os
import json
import asyncio
import datetime

### Import Scripts

In [ ]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Import model index functions
from pyprediktormapclient.model_index import ModelIndex

# Import OPC UA functions
from pyprediktormapclient.opc_ua import OPC_UA

# Import "Dataframer" Tools
from pyprediktormapclient.shared import *

In [ ]:
# Connection to the servers
model_index_url = "http://10.100.59.152:13371/v1/"
opcua_rest_url = "https://apis-opcua-api.pview.dev/"
opcua_server_url = "opc.tcp://81.166.54.193:4853"

# Model index API
model = ModelIndex(url=model_index_url)

# OPC UA API
tsdata = OPC_UA(rest_url=opcua_rest_url, opcua_url= opcua_server_url)

### OPC UA server parameters

In [ ]:
# Input parameters for value data

# Parameters for aggregate historical data
start_time = (datetime.datetime.now() - datetime.timedelta(30)).strftime('%Y-%m-%dT%H:%M:%SZ') # 30 days ago
end_time = (datetime.datetime.now() - datetime.timedelta(29)).strftime('%Y-%m-%dT%H:%M:%SZ') # one day of data
end_time1 = (datetime.datetime.now() - datetime.timedelta(15)).strftime('%Y-%m-%dT%H:%M:%SZ') # 15 days of data
end_time2 = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ') # 30 days of data
pro_interval = 600000 # 10 minutes processing time
agg_name = "Average"

In [ ]:
# All the sites on the OPC server
sites = model.get_objects_of_type('SiteType', return_format="dataframe")
sites

In [ ]:
# Selecting the first site
site = sites.iloc[0]
site_ids = [site['Id']]

INVERTERS DATA

In [ ]:
# All the inverters on the site
inverters = model.get_object_descendants("InverterType", site_ids, "PV_Assets", return_format="dataframe")
inverters

In [ ]:
# List of unique inverter variables
inv_var_list = ['ACActivePower', 'ACCurrentAverage', 'ACCurrentPhaseA', 'ACCurrentPhaseB', 'ACCurrentPhaseC', 'ACDailyActiveEnergy', 'ACReactivePower', 'ACTotalActiveEnergy', 
'ACVoltagePhaseAB', 'ACVoltagePhaseAC', 'ACVoltagePhaseBC', 'DCCurrent', 'DCPower', 'DCVoltage', 'Frequency', 'PowerFactor', 'Status']

In [ ]:
# Live value inverters data
inv_liv_values = tsdata.get_live_values_data(inv_var_list, inverters)
inv_liv_values

In [ ]:
# 1 day aggregated historical inverter data
await tsdata.get_agg_hist_value_data(start_time=start_time, end_time=end_time, pro_interval=pro_interval, agg_name=agg_name, obj_dataframe=inverters, include_variables=inv_var_list)

In [ ]:
# 15 day inverters data
await tsdata.get_agg_hist_value_data(start_time, end_time1, pro_interval, agg_name, inverters, inv_var_list)

In [ ]:
# 1 month inverter data 
await tsdata.get_agg_hist_value_data(start_time, end_time2, pro_interval, agg_name, inverters, inv_var_list)

STRINGS DATA

In [ ]:
# Strings set data 
strings = model.get_object_descendants("StringSetType", site_ids, "PV_Assets", return_format="dataframe")
strings

In [ ]:
# List of strings variables
strng_var_list = ['DCCurrent', 'DCPower', 'DCVoltage', 'RelativeDCPower']

In [ ]:
# Live value strings data 
strng_liv_values = tsdata.get_live_values_data(strng_var_list, strings)
strng_liv_values

In [ ]:
# 1 day strings data
await tsdata.get_agg_hist_value_data(start_time, end_time, pro_interval, agg_name, strings, strng_var_list)

In [ ]:
# 15 days strings data
await tsdata.get_agg_hist_value_data(start_time, end_time1, pro_interval, agg_name, strings, strng_var_list)

In [ ]:
# 1 month strings data
await tsdata.get_agg_hist_value_data(start_time, end_time2, pro_interval, agg_name, strings, strng_var_list)

TRACKERS DATA

In [ ]:
# Trackers data 
trackers = model.get_object_ancestors("TrackerType", get_ids_from_dataframe(strings), "PV_Serves", return_format="dataframe")
trackers

In [ ]:
track_var_list = ['AngleMeasured', 'AngleSetpoint']

In [ ]:
# Live value trackers data
track_liv_values = tsdata.get_live_values_data(track_var_list, trackers)
track_liv_values

In [ ]:
# 1 day trackers data
await tsdata.get_agg_hist_value_data(start_time, end_time, pro_interval, agg_name, trackers, track_var_list)

In [ ]:
# 15 day trackers data 

await tsdata.get_agg_hist_value_data(start_time, end_time1, pro_interval, agg_name, trackers, track_var_list)

In [ ]:
# 1 month trackers data 
await tsdata.get_agg_hist_value_data(start_time, end_time2, pro_interval, agg_name, trackers, track_var_list)